In [12]:
import chromadb
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import SummaryIndex

from llama_index.llms.openai import OpenAI
import os 
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from llama_index.core.objects import ObjectIndex


chroma_client = chromadb.PersistentClient(path="chroma_db")
chroma_collection = chroma_client.get_or_create_collection("quickstart_no_metadata")


In [13]:
function_llm = OpenAI(model="gpt-4",  api_base=os.environ.get("CLOUDFLARE_API_BASE_URL", None))
Settings.llm = OpenAI(model="gpt-4", api_base=os.environ.get("CLOUDFLARE_API_BASE_URL", None))
Settings.embed_model = OpenAIEmbedding(api_base=os.environ.get("CLOUDFLARE_API_BASE_URL", None))

In [14]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
)



In [22]:
engine = index.as_query_engine(llm=Settings.llm)

response = engine.query("summarize in detail,the document related to construction?")
print(response)

The document is a detailed guide for a construction project, outlining the responsibilities of various parties and the proposal submission process. 

The Construction Manager is required to comply with all applicable laws, regulations, and orders, including equal employment opportunity programs. The Owner is responsible for retaining an Architect, covering costs related to environmental studies, surveys, and geotechnical services, and providing all related data to the Construction Manager. The Owner also has a designated representative to assist in decision-making. 

The Construction Manager is required to purchase and maintain insurance as specified in the Construction Contract. The proposal format is outlined in detail, including information about the proposing firm, its organization, personnel, hiring practices, qualifications, experience, sustainability experience, connection to place, self-performed work, subcontracted work, ability to meet timelines, tax-exempt status, and fees. 

In [16]:
response.metadata


{'f0681538-82a2-4a69-ab5e-4d267ab19e8e': {'page_label': '7',
  'file_name': 'kitchenai/5faf76ea-ed89-45ae-b977-16ba61427b25/construction-rfp.pdf',
  'file_id': 33,
  'source': 'kitchenai_cookbook',
  'file_label': 'construction'},
 '6fd2f931-b87f-4844-87bb-d52831ba290c': {'page_label': '6',
  'file_name': 'kitchenai/5faf76ea-ed89-45ae-b977-16ba61427b25/construction-rfp.pdf',
  'file_id': 33,
  'source': 'kitchenai_cookbook',
  'file_label': 'construction'}}

In [17]:

file_label = "construction"

label_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    filter={"file_label": file_label}
)


In [18]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=engine,
        metadata=ToolMetadata(
            name="rfp_collection",
            description=(
                "A catch all tool for any question."
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=label_index.as_query_engine(llm=Settings.llm),
        metadata=ToolMetadata(
            name=file_label,
            description=(
                f"Provides information about the current working RFP: {file_label}"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]




In [19]:
local_agent = OpenAIAgent.from_tools(
    query_engine_tools,
    llm=function_llm,
    verbose=True,
    system_prompt=f"""\
You are an expert in analyzing RFPs and breaking down their requirements from previous incoming RFP requests.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
)


obj_index = ObjectIndex.from_objects(
    query_engine_tools,
    index_cls=VectorStoreIndex,
)

top_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    system_prompt=""" \
You are an RFP expert designed to help the proposal team understand the requirements of incoming RFPs.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\
""",
    verbose=True,
)



In [20]:
response = top_agent.query(f"build a report on the requirements of the incoming RFP: construction, include a list of requirements and a comparison to the advertising and consultancy rfps")


Added user message to memory: build a report on the requirements of the incoming RFP: construction, include a list of requirements and a comparison to the advertising and consultancy rfps
=== Calling Function ===
Calling function: construction with args: {
  "input": "What are the requirements of the incoming RFP: construction?"
}
Got output: The incoming Request for Proposal (RFP) for construction requires the proposer to demonstrate prior experience with Integrated Project / Construction Management-at-risk delivery methods, with at least one project over $8M in construction costs. They should also have prior experience with delivering projects that have achieved sustainability benchmarks such as LEED or Living Building Challenge. The proposer should have qualified personnel with experience collaborating with Architects and Engineers during Design Phases and qualified Cost Estimators on staff. The proposal should include names, addresses, and descriptions of key subcontractors, a deta

In [21]:
response.response

"The incoming RFP for construction has several requirements that differ from the advertising and consultancy RFPs. Here is a comparison:\n\n**Construction RFP:**\n- Requires prior experience with Integrated Project / Construction Management-at-risk delivery methods, with at least one project over $8M in construction costs.\n- Requires prior experience with delivering projects that have achieved sustainability benchmarks such as LEED or Living Building Challenge.\n- Requires qualified personnel with experience collaborating with Architects and Engineers during Design Phases and qualified Cost Estimators on staff.\n- Proposal should include names, addresses, and descriptions of key subcontractors, a detailed summary of current workload and future commitments, and a description of how the firm will manage material purchases to take advantage of the tax-exempt status of the Owner.\n- Proposal should provide the amount of, or basis for, compensation for the Pre-Construction Phase and the Co